In [6]:
from pymongo import MongoClient
import pandas as pd
import seaborn as sns

In [7]:
client = MongoClient()

In [8]:
db = client.amensch
runs = db.runs

In [9]:
res = runs.aggregate([{"$match": {"info.multi_predict_contrast_id": 1}}, {'$project': {'info': 1, 'config': 1}}])

res = list(res)

    

In [18]:
res[0]['config']['n_subjects']

30

In [12]:
multi_dataset_res = runs.aggregate([{"$match": {"info.multi_predict_contrast_id": 1,
                                               "config.datasets": {'$all': [['hcp', 'archi']]}}},
                                    {'$project': {'penalty': '$config.penalty',
                                                  'dropout': '$config.dropout',
                                                  'latent_dim': '$config.latent_dim',
                                                  'alpha': '$config.alpha',
                                                  'activation': '$config.activation',
                                                  'seed': '$config.seed',
                                                  'test_score': '$info.test_archi_score',
                                                  'train_score': '$info.train_archi_score'}}])

multi_dataset_res = list(multi_dataset_res)
df = pd.DataFrame(multi_dataset_res)
df = df.set_index(['activation', 'alpha', 'dropout', 'latent_dim', 'penalty', 'seed'])
df.sort_index(inplace=True)

df_agg = df.groupby(level=['activation', 'alpha', 'dropout', 'latent_dim', 'penalty']).aggregate(['mean', 'std'])['test_score']

argmax = df_agg.loc[df_agg['mean'].argmax()]

df

_id  test_score  \
activation alpha  dropout latent_dim penalty seed                          
linear     0.0001 False   30         l1      386468391   273    0.850000   
                                             1362340733  275    0.845833   
                                             1545808924  274    0.800000   
                                             2351619344  276    0.854167   
                                             4201173489  272    0.887500   
                                     l2      386468391   278    0.841667   
                                             1362340733  280    0.845833   
                                             1545808924  279    0.808333   
                                             2351619344  281    0.841667   
                                             4201173489  277    0.866667   
                          100        l1      386468391   283    0.850000   
                                             1362340733  285    0.870833   
                                             1545808924  284    0.791667   
                                             2351619344  286    0.858333   
                                             4201173489  282    0.870833   
                                     l2      386468391   288    0.862500   
                                             1362340733  290    0.879167   
                                             1545808924  289    0.800000   
                                             2351619344  291    0.845833   
                                             4201173489  287    0.870833   
                          200        l1      386468391   293    0.858333   
                                             1362340733  295    0.870833   
                                             1545808924  294    0.795833   
                                             2351619344  296    0.883333   
                                             4201173489  292    0.858333   
                                     l2      386468391   298    0.845833   
                                             1362340733  300    0.870833   
                                             1545808924  299    0.812500   
                                             2351619344  301    0.845833   
                                             4201173489  297    0.870833   
...                                                      ...         ...   
relu       0.1000 True    30         l1      386468391   183    0.066667   
                                             1362340733  185    0.025000   
                                             1545808924  184    0.079167   
                                             2351619344  186    0.033333   
                                             4201173489  182    0.050000   
                                     l2      386468391   188    0.033333   
                                             1362340733  190    0.033333   
                                             1545808924  189    0.033333   
                                             2351619344  191    0.033333   
                                             4201173489  187    0.033333   
                          100        l1      386468391   193    0.079167   
                                             1362340733  195    0.070833   
                                             1545808924  194    0.191667   
                                             2351619344  196    0.141667   
                                             4201173489  192    0.045833   
                                     l2      386468391   198    0.033333   
                                             1362340733  200    0.033333   
                                             1545808924  199    0.033333   
                                             2351619344  201    0.033333   
                                             4201173489  197    0.033333   
                          200

In [13]:
len(multi_dataset_res)

480

In [14]:
df_agg.loc[df_agg['mean'].argmax()]

mean    0.872500
std     0.038595
Name: (relu, 0.0001, True, 200, l1), dtype: float64

In [15]:
simple_dataset_res = runs.aggregate([{"$match": {"info.multi_predict_contrast_id": 1,
                                                 "config.datasets": {"$not": {"$all": ['archi','hcp']}},
                                                 'config.factored': True
                                                }},
                                    {'$project': {'penalty': '$config.penalty',
                                                  'dropout': '$config.dropout',
                                                  'latent_dim': '$config.latent_dim',
                                                  'alpha': '$config.alpha',
                                                  'activation': '$config.activation',
                                                  'seed': '$config.seed',
                                                  'test_score': '$info.test_archi_score',
                                                  'train_score': '$info.train_archi_score'}}])
simple_dataset_res = list(simple_dataset_res)
df_s = pd.DataFrame(simple_dataset_res)    
df_s = df_s.set_index(['activation', 'alpha', 'dropout', 'latent_dim', 'penalty', 'seed'])
df_s.sort_index(inplace=True)

df_s_agg = df_s.groupby(level=['activation', 'alpha', 'dropout', 'latent_dim', 'penalty']).aggregate(['mean', 'std'])['test_score']

argmax = df_s_agg.loc[df_s_agg['mean'].argmax()]

In [17]:
df_s - df

_id  test_score  \
activation alpha  dropout latent_dim penalty seed                          
linear     0.0001 False   30         l1      386468391   480    0.000000   
                                             1362340733  480    0.000000   
                                             1545808924  480    0.000000   
                                             2351619344  480    0.000000   
                                             4201173489  480    0.000000   
                                     l2      386468391   480    0.000000   
                                             1362340733  480    0.000000   
                                             1545808924  480    0.000000   
                                             2351619344  480    0.000000   
                                             4201173489  480    0.000000   
                          100        l1      386468391   480    0.000000   
                                             1362340733  480    0.000000   
                                             1545808924  480    0.000000   
                                             2351619344  480    0.000000   
                                             4201173489  480    0.000000   
                                     l2      386468391   480    0.000000   
                                             1362340733  480    0.000000   
                                             1545808924  480    0.000000   
                                             2351619344  480    0.000000   
                                             4201173489  480    0.000000   
                          200        l1      386468391   480    0.000000   
                                             1362340733  480    0.000000   
                                             1545808924  480    0.000000   
                                             2351619344  480    0.000000   
                                             4201173489  480    0.000000   
                                     l2      386468391   480    0.000000   
                                             1362340733  480    0.000000   
                                             1545808924  480    0.000000   
                                             2351619344  480    0.000000   
                                             4201173489  480    0.000000   
...                                                      ...         ...   
relu       0.1000 True    30         l1      386468391   480   -0.016667   
                                             1362340733  480    0.125000   
                                             1545808924  480   -0.050000   
                                             2351619344  480    0.000000   
                                             4201173489  480    0.000000   
                                     l2      386468391   480    0.000000   
                                             1362340733  480    0.000000   
                                             1545808924  480    0.000000   
                                             2351619344  480    0.000000   
                                             4201173489  480    0.000000   
                          100        l1      386468391   480    0.000000   
                                             1362340733  480    0.175000   
                                             1545808924  480    0.000000   
                                             2351619344  480    0.000000   
                                             4201173489  480    0.020833   
                                     l2      386468391   480    0.000000   
                                             1362340733  480    0.000000   
                                             1545808924  480    0.000000   
                                             2351619344  480    0.000000   
                                             4201173489  480    0.000000   
                          200

In [299]:
df - df_s

_id  test_score  \
activation alpha  dropout latent_dim penalty seed                          
linear     0.0001 False   30         l1      386468391  -480    0.000000   
                                             1362340733 -480    0.000000   
                                             1545808924 -480    0.000000   
                                             2351619344 -480    0.000000   
                                             4201173489 -480    0.000000   
                                     l2      386468391  -480    0.000000   
                                             1362340733 -480    0.000000   
                                             1545808924 -480    0.000000   
                                             2351619344 -480    0.000000   
                                             4201173489 -480    0.000000   
                          100        l1      386468391  -480    0.000000   
                                             1362340733 -480    0.000000   
                                             1545808924 -480    0.000000   
                                             2351619344 -480    0.000000   
                                             4201173489 -480    0.000000   
                                     l2      386468391  -480    0.000000   
                                             1362340733 -480    0.000000   
                                             1545808924 -480    0.000000   
                                             2351619344 -480    0.000000   
                                             4201173489 -480    0.000000   
                          200        l1      386468391  -480    0.000000   
                                             1362340733 -480    0.000000   
                                             1545808924 -480    0.000000   
                                             2351619344 -480    0.000000   
                                             4201173489 -480    0.000000   
                                     l2      386468391  -480    0.000000   
                                             1362340733 -480    0.000000   
                                             1545808924 -480    0.000000   
                                             2351619344 -480    0.000000   
                                             4201173489 -480    0.000000   
...                                                      ...         ...   
relu       0.1000 True    30         l1      386468391  -480    0.016667   
                                             1362340733 -480   -0.125000   
                                             1545808924 -480    0.050000   
                                             2351619344 -480    0.000000   
                                             4201173489 -480    0.000000   
                                     l2      386468391  -480    0.000000   
                                             1362340733 -480    0.000000   
                                             1545808924 -480    0.000000   
                                             2351619344 -480    0.000000   
                                             4201173489 -480    0.000000   
                          100        l1      386468391  -480    0.000000   
                                             1362340733 -480   -0.175000   
                                             1545808924 -480    0.000000   
                                             2351619344 -480    0.000000   
                                             4201173489 -480   -0.020833   
                                     l2      386468391  -480    0.000000   
                                             1362340733 -480    0.000000   
                                             1545808924 -480    0.000000   
                                             2351619344 -480    0.000000   
                                             4201173489 -480    0.000000   
                          200

In [300]:
multinomial_res = runs.aggregate([{"$match": {"info.multi_predict_contrast_id": 1,
                                               "config.datasets": 'archi',
                                                 'config.factored': False
                                                }},
                                    {'$project': {'penalty': '$config.penalty',
                                                  'alpha': '$config.alpha',
                                                  'seed': '$config.seed',
                                                  'test_score': '$info.test_archi_score',
                                                  'train_score': '$info.train_archi_score'}}])

df_m = pd.DataFrame(list(multinomial_res))
df_m = df_m.set_index(['alpha', 'penalty', 'seed'])
df_m.sort_index(inplace=True)

df_m_agg = df_m.groupby(level=['alpha', 'penalty']).aggregate(['mean', 'std'])['test_score']

argmax = df_m_agg['mean'].argmax()

print(argmax)

df_m_agg.loc[argmax]

(0.0001, 'l2')


mean    0.856667
std     0.029698
Name: (0.0001, l2), dtype: float64

In [301]:
df_m

_id  test_score  train_score
alpha  penalty seed                                     
0.0001 l1      386468391    963    0.854167     0.984127
               1362340733   965    0.845833     0.985714
               1545808924   964    0.795833     0.986772
               2351619344   966    0.875000     0.980952
               4201173489   962    0.866667     0.977249
       l2      386468391    968    0.870833     0.985714
               1362340733   970    0.875000     0.987302
               1545808924   969    0.804167     0.985714
               2351619344   971    0.862500     0.982540
               4201173489   967    0.870833     0.975661
0.0010 l1      386468391    973    0.754167     0.808466
               1362340733   975    0.787500     0.810582
               1545808924   974    0.729167     0.823810
               2351619344   976    0.787500     0.808995
               4201173489   972    0.787500     0.811640
       l2      386468391    978    0.837500     0.913228
               1362340733   980    0.862500     0.919048
               1545808924   979    0.795833     0.923280
               2351619344   981    0.862500     0.908466
               4201173489   977    0.858333     0.908995
0.0100 l1      386468391    983    0.166667     0.212169
               1362340733   985    0.183333     0.214815
               1545808924   984    0.187500     0.221693
               2351619344   986    0.187500     0.209524
               4201173489   982    0.195833     0.205291
       l2      386468391    988    0.695833     0.767196
               1362340733   990    0.737500     0.778307
               1545808924   989    0.683333     0.784656
               2351619344   991    0.762500     0.769312
               4201173489   987    0.712500     0.768254
0.1000 l1      386468391    993    0.033333     0.033333
               1362340733   995    0.033333     0.033333
               1545808924   994    0.033333     0.033333
               2351619344   996    0.033333     0.033333
               4201173489   992    0.000000     0.000000
       l2      386468391    998    0.395833     0.407937
               1362340733  1000    0.391667     0.424339
               1545808924   999    0.391667     0.425397
               2351619344  1001    0.370833     0.426455
               4201173489   997    0.408333     0.408995